In [4]:
# Parameters
input_data_path = ""
hvg = ""
batch = ""
label = ""


Jupyter notebook for conos integration algoritm

Author: Erno Hänninen

Title: run_conos.ipynb

Created: 2022-11-29

In [5]:
require(conos)
require(Seurat)
require(rhdf5)

In [ ]:
sobj = readRDS(input_data_path)
sobj

In [7]:
#This function is from https://github.com/theislab/scib-pipeline/blob/0c7be53b1000864fcd31a7b7594f9a5071204233/scripts/integration/integration.R
preP <- function(so, vars.to.regress=NULL, verbose=TRUE, n.pcs=100) {
  if (verbose) {
    message("Running Seurat v3 workflow")
  }
  so <- Seurat::FindVariableFeatures(object = so, verbose = verbose)
  so <- Seurat::ScaleData(object = so, verbose = verbose)
  so <- Seurat::RunPCA(object = so, npcs = n.pcs, verbose = verbose)
  return(so)
}


In [ ]:
#This function is from https://github.com/theislab/scib-pipeline/blob/0c7be53b1000864fcd31a7b7594f9a5071204233/scripts/integration/integration.R
runConos = function(sobj, batch) {
    batch_list <- SplitObject(sobj, split.by=batch)
    pp <- lapply(batch_list, preP)
    con <- Conos$new(pp)
    con$buildGraph(space="genes")
    con$findCommunities()
    con$embedGraph(method="UMAP")
    
    return(con)
}


In [ ]:
if(file.exists(hvg)) {
	print("EXISTS")
	hvg<-unlist(readRDS(hvg), use.names=FALSE)
	print(length(hvg))
	if(length(hvg) > 0){
		sobj <- subset(sobj, features=hvg)
	}
} 

con=runConos(sobj, batch)

In [ ]:
if (file.exists("../Integrated_adata/conos_adata.h5")) {
  #Delete file if it exists
  file.remove("../Integrated_adata/conos_adata.h5")
}
meta <- function(sobj) {return(sobj@meta.data)}
metalist <- lapply(con$samples, meta)
metaM <- do.call(rbind,unname(metalist))
saveConosForScanPy(con, output.path="../Integrated_adata/", hdf5_filename="conos_adata.h5", pseudo.pca=TRUE, pca=TRUE, verbose=TRUE, metadata.df=metaM)